 # Safety
Safety concerns are a major reason why nuclear energy remains politically divisive and less widely adopted. Incidents like Chernobyl and Fukushima highlight the potential risks. In this section, we will examine nuclear energy safety by analysing historical incidents, overall death rates, and the management of nuclear waste.

<!-- One of the primary reasons nuclear energy has not been widely implemented and remains politically divisive is safety concerns. These safety concerns are not unfounded. Well known nuclear energy incidents like Chernobyl and Fukushima, which were catastrophic in their region, are often cited as reasons why widely implementing nuclear energy is a damning suggestion. 

In this section we will analyse the safety of nuclear energy looking at nuclear incidents, general death-rate, and nuclear waste.  --> 

<!-- 

<!-- It is often assumed that countries with higher reliance on nuclear energy are better prepared for nuclear incidents due to increased safety protocols and experience handling the technology. When analysing our dataset, we found that this assumption holds true in some regions but not universally.

When looking at the percentage of electricity generated by nuclear energy between 2000 and 2020, we observed that countries like France and several in Northern Europe consistently displayed high nuclear dependency—often reaching well over 70% of their total electricity mix. This long-standing reliance on nuclear power reflects both national energy policies and historical investments in nuclear infrastructure: -->
## Nuclear energy percentage per country and their incidents

It is often assumed that countries with a high dependence on nuclear energy are better prepared for nuclear incidents due to strict safety protocols and greater operational experience. When analysing our dataset, this assumption appears true in some regions but not universally.

Between 2000 and 2020, countries like France and several Northern European nations showed high nuclear reliance, often exceeding 70% of their electricity mix. This reflects historical investments and strong national energy policies.

However, globally, incident data paints a more nuanced picture. Countries with low nuclear output, like Ukraine and Japan, have still experienced the most severe accidents, suggesting that infrastructure quality, regulation, and historical reactor designs play a bigger role in safety than output levels alone.



In [11]:
%run nuclear_energy_incidents.ipynb


<!-- 
By comparison, the vast majority of countries, especially in Africa, the Middle East, and parts of Asia, remained largely dependent on fossil fuels or renewables, with nuclear contributing little to nothing to their energy production. These nations are represented in the darker red regions of the graph, where the percentage of nuclear electricity remains close to zero throughout the twenty-year period.

However, the global distribution of nuclear incidents tells a more complex story. The incident markers on the map, scaled by the number of direct and indirect deaths, show that catastrophic events are not exclusive to countries with the highest nuclear output. Major accidents such as Chernobyl (Ukraine) and Fukushima (Japan) stand out, both in marker size and location, reflecting their high death tolls and severity ratings on the INES scale.

Interestingly, there is little correlation between a country’s reliance on nuclear energy and its incident frequency or severity. For example, while France maintains high nuclear use, it shows a relatively low number of deadly incidents compared to countries like Ukraine or Japan. This could suggest that policy maturity, regulatory oversight, and historical factors (such as Soviet-era reactor design in the case of Chernobyl) play a larger role in incident likelihood than energy output alone.

It should also be noted that the time-based slider in the visualization reveals no major expansion in nuclear electricity production globally after 2011, the year of the Fukushima disaster. This aligns with research by Sovacool (2011), who noted a significant decline in nuclear energy investment and public support following Fukushima. In fact, countries like Germany actively reduced their nuclear energy share post-2011, a policy shift clearly visible in the color change on the map during later years.

Nevertheless, while incident history can impact national policy and public perception, the long-term reliance on nuclear power continues in certain regions, often driven by goals of energy independence and decarbonization. The map illustrates this ongoing tension: between the benefits of low-carbon nuclear energy and the shadow of catastrophic risk that continues to shape global nuclear discourse. -->



## Nuclear energy death-rate

In this section we analyse the total death-rate of different energy sources. This section ties into perspective 2. Consider the following bar graph:

In [12]:
%run death_rate.ipynb

In the bar graph we see that nuclear, wind and solar are the safest forms of energy production in terms of death rate. The death rate for wind, nuclear, and solar are 0.035, 0.030, and 0.019 respectively. This orders of magnitude smaller than the fossil fuels. 


## Nuclear Waste

An important aspect of nuclear energy is nuclear waste. Nuclear reactors produce highly radioactive waste that needs to be stored for decades on end. In this section we will focus on radioactive waste and how the creation of radioactive waste is increasing. The data in this section is from the **U.S. Energy Information Administration**. This means that this section does not represent the whole globe but only the **U.S.**.

### Average Burnup

The 'burnup' of a nuclear reactor is a term for the efficiency of a reactor in GigaWatt days per metric ton of uranium (GWd / MTU). The first graph shows the average burnup of powerplants in the U.S. through time.

In [ ]:
%run burnup.ipynb


We see that the efficiency of the reactors increases from the 1970s tot the mid 2000s. In the mid 2000s however, the efficiency seems to plateu around 45 GWd/MTU

## Summary

- Major fatal incidents are extremly rare, even in countries where nuclear energy has been widely implemented. Most incidents happened well over 20 years ago and produce (close to) zero casualties.

- The death rate of nuclear energy is among the lowest of all energy sources, among renewables like wind and solar. It is orders of magnitude lower than that of fossil fuels

- The efficiency of nuclear reactors seem to have reach a plateau in the mid 2000s at around 45 GWd/MTU. 

Nuclear energy is a safe form of energy. By many measures safer than traditional fossil fuels. Although the memories of nuclear incidents such as Chernobyl might paint a dark picture, modern safety standards are extremly high and this shows in the death rate and incidents. 